# Obtenção dos Dados:

Este projeto foi inspirado pelas atividades previamente realizadas por [Kizzy Terra](#https://github.com/kterra) e os dados utilizados estão neste [repositório](#https://github.com/kterra/ciencia-de-dados-aplicada), que segundo a própria autora foram dados obtidos pelo PRODES.

_"O projeto PRODES realiza o monitoramento por satélites do desmatamento por corte raso na Amazônia Legal e produz, desde 1988, as taxas anuais de desmatamento na região, que são usadas pelo governo brasileiro para o estabelecimento de políticas públicas."_ - [Coordenação Geral de Observação da Terra INPE](#http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes#:~:text=O%20projeto%20PRODES%20realiza%20o,o%20estabelecimento%20de%20pol%C3%ADticas%20p%C3%BAblicas.)

In [127]:
from modules import *

# DATASETS = [pd.read_csv(f'../Data/DesmatamentoMunicipios20{digits}.txt', sep=',', encoding='iso-8859-1') for digits in range(0,18)]
DATASETS = []

for i in range(0,19):
    a = '../Data/2000_2018/DesmatamentoMunicipiosAM2000.txt'
    if i < 10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM200{i}.txt', sep=',', encoding='iso-8859-1'))
    elif i>=10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM20{i}.txt', sep=',', encoding='iso-8859-1'))
     

type(DATASETS[0])


pandas.core.frame.DataFrame

**DATASETS** é uma lista com 19 dataframes com as informações dos arquivos que estão no diretório _`../Data/2000_2018`_

In [128]:
DATASETS[0].sample(3)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2000,Incremento19992000,Floresta2000,Nuvem2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Soma
39,40,-7.61678,-70.19515,S 7 37 0.41,O 70 11 42.54,Envira,1301506,AM,7793,138.8,NaN,7586.0,0.0,5.3,0.1,62.8,100
35,36,-7.14334,-71.42004,S 7 8 36.02,O 71 25 12.14,Ipixuna,1301803,AM,12595,149.7,NaN,12307.9,0.0,0.6,66.6,70.2,100
59,60,-3.47332,-61.67090,S 3 28 23.95,O 61 40 15.24,Anamã,1300086,AM,2473,177.0,NaN,2042.4,0.0,0.0,34.6,219.0,100


In [129]:
DATASETS[1].sample(2)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2001,Incremento20002001,Floresta2001,Nuvem2001,NaoObservado2001,NaoFloresta2001,Hidrografia2001,Soma
21,22,-5.24148,-58.19462,S 5 14 29.33,O 58 11 40.63,Maués,1302900,AM,40090,1151.4,22.4,36769.2,0.0,2.2,872.3,1294.9,100
30,31,-1.38598,-68.79404,S 1 23 9.53,O 68 47 38.54,Japurá,1302108,AM,57555,45.3,1.8,55886.7,1.9,98.3,2.6,1520.2,99


**MunicipalAdjascencyDict** é um dicionário contendo como chave o nome de uma cidade e como valor os municípios de adjascência da mesma

A referência de Ajascência municipal segue o modelo de mapa proposto pelo [Mapa do Escolar feito pelo IBGE](#https://geoftp.ibge.gov.br/produtos_educacionais/mapas_tematicos/mapas_do_brasil/mapas_estaduais/politico/amazonas.jpg) modificado em 2016, segue o exemplo:


In [130]:
MunicipalAdjascencyDict = {
'Urucurituba' : 
['Boa Vista do Ramos', 'Barreirinha', 'Itapiranga', 'Itacoatiara', 'Parintins', 'Urucara', 'Silves', 'Nhamunda'],

'Urucara' : 
['Nhamunda', 'Urucurituba', 'Itapiranga', 'Sao Sebastiao do Uatuma', 'Presidente Figueiredo'],

'Uarini' : 
['Manaa','Fonte Boa','Jurua', 'Alvaraes','Tefe'],

'Tonantins' : 
['Japura', 'Santo Antonio do Iça', 'Fonte Boa', 'Jutai'],

'Tefe' : 
['Tapaua', 'Manaa', 'Carauari', 'Jurua', 'Alvaraes', 'Uarini'],

'Tapaua' : 
['Itamarati', 'Pauini', 'Labrea', 'Canutama', 'Carauari', 'Tefe', 'Coari', 'Anori', 'Beruri', 'Manicore', 'Humaita'],

'Tabatinga' : 
['Benjamin Constant', 'Santo Antonio do Iça', 'Sao Paulo de Olivença'],

'Silves' : 
[ 'Urucurituba', 'Itacoatiara', 'Itapiranga'],

'Sao Sebastiao do Uatuma' : 
['Urucura', 'Presidente Figueiredo', 'Itapiranga'],

'Sao Paulo de Olivenca' : 
['Benjamin Constant', 'Jutai', 'Tabatinga', 'Santo Antonio do Iça', 'Amatur'],

'Sao Gabriel de Cachoeira' : 
['Japura', 'Santa Isabel do Rio Negro'],

'Santo Antonio do Ica' : 
['Sao Paulo de Olivença', 'Tabatinga', 'Amatur' , 'Tonantins', 'Jutai', 'Japura'],

'Santa Isabel do Rio Negro' : 
['Sao Gabriel da Cachoeira', 'Japura', 'Manaa', 'Barcelos'],

'Rio Preto da Eva' : 
['Itacoatiara', 'Manaus', 'Itapiranga', 'Presidente Figueiredo'],

'Presidente Figueiredo' : 
['Rio Preto de Eva', 'Manaus', 'Novo Airao', 'Itapiranga', 'Sao Sebastiao do Uatuma', 'Urucua'],

'Pauini' : #15
['Boca do Acre', 'Envira', 'Itamarati', 'Eirunepe','Tapaua', 'Labrea'],

'Parintins' : #16
['Nhamunda', 'Urucurituba', 'Barreirinha'],

'Novo Aripuana' : 
['Manicore', 'Apui', 'Borba'],

'Novo Airao' :  #18
['Barcelos', 'Codajas', 'Caapiranga', 'Manacapuru', 'Manaus', 'Presidente Figueiredo'],

'Nova Olinda do Norte' : 
['Borba', 'Maues', 'Autazes', 'Itacoatiara'],

'Nhamunda' :  
['Urucura', 'Urucurituba', 'Parintins', 'Maues'],

'Maues' : #21
['Apui', 'Borba', 'Itacoatiara', 'Boa Vista do Ramos', 'Barreirinha', 'Nova Olinda do Norte'],

'Manaa' : 
['Fonte boa', 'Japura', 'Santa Izabel do Rio Negro', 'Barcelos', 'Codajas', 'Coari', 'Tefe', 'Uarini'],

'Manicore' : 
['Humaita', 'Tapaua', 'Beruri', 'Borba','Novo Aripuana'],

'Manaus' : 
['Manacapuru','Itacoatiara','Careiro', 'Careiro da Varzea', 'Iranduba', 'Manacapuru', 'Novo Airao', 'Presidente Figueiredo', 'Rio Preto da Eva'],

'Manaquiri' : 
['Manacapuru', 'Borba', 'Beruri', 'Iranduba', 'Manaus', 'Careiro'],

'Manacapuru' : 
['Novo Airao', 'Caapiranga', 'Anama', 'Anori', 'Beruri', 'Manaquiri', 'Iranduba', 'Manaus'],

'Labrea' : 
['Boca do Acre', 'Pauini', 'Itamarati', 'Tapaua', 'Canutama'],

'Jutai' :  
['Itamarati', 'Eirunepe', 'Benjamin Constant', 'Sao Paulo de Olivença','Amatur','Santo Antonio do Iça', 'Tonantins', 'Fonte Boa', 'Jurua', 'Carauari'],

'Jurua' :  
['Carauari', 'Jutai', 'Fonte Boa', 'Uarini', 'Alvaraes', 'Tefe'],

'Japura' : 
['Santo Antonio de Iça',' Tonantins','Fonte Boa', 'Manaa','Santa Isabel do Rio Negro',' Sao Gabriel de Cachoeira'],

'Itapiranga' :  
['Presidente Figueiredo','Sao Sebastiao do Uatuma','Urucura','Urucurituba', 'Silves', 'Rio Preto de Eva', 'Itacoatiara'],

'Itamarati' : 
['Envira', 'Eirunepe', 'Jutai', 'Carauari', 'Tapua','Labrea','Pauini'],

'Itacoatiara' : 
['Autazes', 'Careiro da Varzea', 'Maues', 'Rio Preto da Eva', 'Nova Olinda do Norte', 'Manaus', 'Silves', 'Itapiranga', 'Urucurituba', 'Boa Vista do Ramos'],

'Iranduba' :  
['Novo Airao','Careiro', 'Careiro da Varzea', 'Manacapuru', 'Manaquiri', 'Manaus'],

'Ipixuna' : 
['Eirunepe', 'Benjamin Constant', 'Atalaia do Norte', 'Guajara'],

'Humaita' : 
['Canutama', 'Tapaua', 'Manicore'],

'Guajara' : 
['Atalaia do Norte', 'Ipixuna'],

'Fonte Boa' : 
['Jutai', 'Jurua', 'Tonantins', 'Japura', 'Manaa', 'Uarini'],

'Envira' : 
['Pauini', 'Itamarati', 'Eirunepe'],

'Eirunepe' : 
['Benjamin Constant', 'Ipixuna', 'Envira', 'Itamarati', 'Jutai','Pauini'],

'Codajas' : 
['Anama','Anori','Coari','Manaa','Barcelos','Novo Airao','Caapiranga'],

'Coari' : 
['Manaa', 'Codajas', 'Tefe', 'Tapua', 'Anori','Barcelos'],

'Careiro da Varzea' : 
['Iranduba', 'Careiro', 'Manaus', 'Itacoatiara', 'Autazes'],

'Careiro' : 
['Iranduba', 'Manacapuru', 'Manaquiri', 'Borba', 'Autazes', 'Careiro da Varzea', 'Manaus'],

'Carauari' : 
['Tefe', 'Alvaraes', 'Jurua', 'Jutai', 'Itamarati', 'Tapua'],

'Canutama' :  
['Labrea', 'Tapaua', 'Humaita'],

'Caapiranga' : 
['Anama', 'Manacapuru', 'Novo Airao', 'Codajas'],

'Borba' : 
['Maues', 'Nova Olinda do Norte', 'Autazes', 'Careiro', 'Manaquiri', 'Beruri', 'Manicore', 'Novo Aripuana', 'Apui'],

'Boca do Acre' : 
['Pauini', 'Labrea'],

'Boa Vista do Ramos' : 
['Maues','Barreirinha', 'Urucurituba', 'Itacoatiara'],

'Beruri' : 
['Manicore', 'Tapua', 'Anori', 'Anama', 'Manacapuru', 'Manaquiri'],

'Benjamin Constant' : 
['Atalaia do Norte', 'Ipixuna', 'Eirunepe', 'Jutai', 'Sao Paulo de Olivença', 'Tabatinga'],

'Barreirinha' : 
['Maues', 'Boa Vista do Ramos', 'Urucurituba', 'Parintins'],

'Barcelos' : 
['Santa Isabel do Rio Negro', 'Novo Airao', 'Manaa', 'Codajas', 'Coari'],

'Autazes' : 
['Manaus', 'Careiro', 'Careiro da Varzea', 'Borba', 'Itacoatiara', 'Nova Olinda do Norte'],

'Atalaia do Norte' :  
['Benjamim Constant', 'Guajara','Ipixuna'],

'Apui' : 
['Novo Aripuana', 'Borba', 'Maues'],

'Anori' : 
['Beruri', 'Tapua', 'Coari', 'Codajas', 'Anama'],

'Anama' :  
['Manacapuru', 'Beruri',' Anori','Codajas','Caapiranga'],

'Amatur' : 
['Sao Paulo de OLivença', 'Jutai', 'Santo Antonio do Iça'],

'Alvaraes' : 
['Tefe', 'Carauari', 'Jurua', 'Uarini'],
}

# Limpeza e Inserção de Dados:

A remoção de caracteres e de colunas não utilizadas facilitará o trabalho com as informações essênciais. Além disso a normalização dos dados e estabelecimento de adjascência entre municípios será essêncial para análise destes e modelagem dos grafos.

### Removendo colunas irrelevantes:

In [131]:
print(DATASETS[0].columns)

Index(['Nr', 'Lat', 'Long', 'Latgms', 'Longms', 'Municipio', 'CodIbge',
       'Estado', 'AreaKm2', 'Desmatado2000', 'Incremento19992000',
       'Floresta2000', 'Nuvem2000', 'NaoObservado2000', 'NaoFloresta2000',
       'Hidrografia2000', 'Soma '],
      dtype='object')


In [132]:
DATASETS[0].drop(columns = ['Incremento19992000'], inplace = True)

In [133]:

DATASETS[0]

for index, value in enumerate(DATASETS):
    if index < 10:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem200{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Soma '], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)
        
    else:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem20{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)

DATASETS[0].sample(2)

,Lat,Long,Municipio,AreaKm2,Desmatado2000,Floresta2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000
1,-1.24938,-58.78714,Urucará,28012,299.2,26642.5,0.9,423.0,646.4
45,-5.17242,-67.49476,Carauari,26469,258.2,25791.1,7.1,12.7,399.9


### Tirando acento dos caracteres

In [134]:
#Criando método para retirar os acentos das palavras dentro deste dataset:
def substituir_caracteres(columns):  
  columns = columns.replace('Á','A').replace('Â','A').replace('Ã','A').replace('À','A').replace('É','E').replace('Ê','E').replace('È','E').replace('Í','I').replace('Ì','I').replace('Î','I').replace('Ó','O').replace('Ò','O').replace('Õ','O').replace('Ô','O').replace('Ú','U').replace('Ù','U').replace('Û','U').replace('Ç','C').replace('á','a').replace('â','a').replace('ã','a').replace('à','a').replace('é','e').replace('ê','e').replace('è','e').replace('í','i').replace('ì','i').replace('î','i').replace('ó','o').replace('ò','o').replace('õ','o').replace('ô','o').replace('ú','u').replace('ù','u').replace('û','u').replace('ç','c').replace('-', ' ').replace('.','')
  return columns

for index,dataset in enumerate(DATASETS):
  DATASETS[index]['Municipio'] = DATASETS[index]['Municipio'].apply(substituir_caracteres)

DATASETS[0]['Municipio'].sample(15)

53                  Barreirinha
23                     Manicore
38                    Fonte Boa
14        Presidente Figueiredo
25                    Manaquiri
12    Santa Isabel do Rio Negro
20                     Nhamunda
28                        Jutai
51                       Beruri
29                        Jurua
42                        Coari
36                      Humaita
57                         Apui
52            Benjamin Constant
13             Rio Preto da Eva
Name: Municipio, dtype: object

### Criando coluna de fronteira entre municípios para todos os datasets

In [135]:

MunicipalAdjascencyList = [x for x in MunicipalAdjascencyDict.values()]

for i in DATASETS:
    i['Fronteiras'] = MunicipalAdjascencyList

DATASETS[0].sample(4)

,Lat,Long,Municipio,AreaKm2,Desmatado2000,Floresta2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Fronteiras
40,-7.04074,-70.12601,Eirunepe,15608,287.1,14961.8,5.3,172.1,181.7,"[Benjamin Constant, Ipixuna, Envira, Itamarati..."
9,-4.73714,-69.52392,Sao Paulo de Olivenca,20467,103.8,19688.8,0.3,86.3,587.8,"[Benjamin Constant, Jutai, Tabatinga, Santo An..."
6,-4.02878,-69.69786,Tabatinga,3343,137.0,3123.1,29.6,1.3,52.0,"[Benjamin Constant, Santo Antonio do Iça, Sao ..."
2,-3.20648,-65.43058,Uarini,10448,194.2,9721.7,0.0,100.3,431.8,"[Manaa, Fonte Boa, Jurua, Alvaraes, Tefe]"


### Estabelecendo média de desmatamento:

In [136]:
# mean_column = []

mean = DATASETS[0]['Desmatado2000'].mean()

# print(mean)
for index, dataframe in enumerate(DATASETS):
    mean_column = []
    if index < 10:
        mean = dataframe[f'Desmatado200{index}'].mean()
        for i in dataframe[f'Desmatado200{index}']:
            mean_column.append(True) if i > mean else mean_column.append(False)
    else:
        mean = dataframe[f'Desmatado20{index}'].mean()
        for i in dataframe[f'Desmatado20{index}']:
            mean_column.append(True) if i > mean else mean_column.append(False)

    DATASETS[index]['MaiorQueAMédia'] = mean_column

print(DATASETS[2]['Desmatado2002'].mean())
DATASETS[2].sample(6)
# exit()

        
#    if DATASETS[0]['Desmatado2000'].mean() < 

451.51290322580644


,Lat,Long,Municipio,AreaKm2,Desmatado2002,Incremento20012002,Floresta2002,NaoObservado2002,NaoFloresta2002,Hidrografia2002,Fronteiras,MaiorQueAMédia
8,-1.89625,-58.81001,Sao Sebastiao do Uatuma,10776,171.6,2.9,9912.4,0.5,65.7,625.8,"[Urucura, Presidente Figueiredo, Itapiranga]",False
33,-3.13511,-58.81276,Itacoatiara,8927,1580.6,56.1,4851.0,0.5,829.3,1665.6,"[Autazes, Careiro da Varzea, Maues, Rio Preto ...",True
45,-5.17242,-67.49476,Carauari,26469,277.2,9.2,25772.1,7.1,12.7,399.9,"[Tefe, Alvaraes, Jurua, Jutai, Itamarati, Tapua]",False
59,-3.47332,-61.67090,Anama,2473,189.6,1.3,2029.8,0.0,34.6,219.0,"[Manacapuru, Beruri, Anori, Codajas, Caapiranga]",False
25,-3.73889,-60.65552,Manaquiri,4001,475.2,2.0,3132.4,0.0,106.3,287.1,"[Manacapuru, Borba, Beruri, Iranduba, Manaus, ...",True
6,-4.02878,-69.69786,Tabatinga,3343,146.5,3.8,3113.6,29.6,1.3,52.0,"[Benjamin Constant, Santo Antonio do Iça, Sao ...",False
